In [1]:
import shutil
from pathlib import Path
import flammkuchen as fl
from skimage import io
from split_dataset import SplitDataset
import numpy as np
from fimpy.pipeline.alignment import align_2p_volume
from fimpy.pipeline.general import make_anatomy
from scipy.ndimage import affine_transform

def transl_mx(shift):
    return np.array([[1,0,shift[0]], [0,1,shift[1]], [0,0,1]])

def rot_mx(theta):
    return np.array([[np.cos(theta),-np.sin(theta),0], [np.sin(theta),np.cos(theta),0], [0,0,1]])

In [2]:
path = Path(r'C:\Users\You\Documents\test')

In [3]:
src = SplitDataset(path/'original')
aligned = align_2p_volume(src, across_planes=False, n_jobs=3)

Calculating filtered reference
Aligning individual planes...


In [4]:
src = SplitDataset(path/'original')
aligned = sorted(list((path/'aligned').glob('*.h5')))
rot_theta = fl.load(path/'rot_theta.h5')
output_dir = path/"rotated"
output_dir.mkdir()
shutil.copy(path/"aligned/stack_metadata.json", output_dir/"stack_metadata.json")

for plane in range(src.shape[1]):
    theta = rot_theta[plane]
    if theta == 0:
        shutil.copy(path/"aligned/{:04d}.h5".format(plane), output_dir/"{:04d}.h5".format(plane))
    else:
        data = src[:, plane:plane+1, :, :]
        shifts = fl.load(aligned[plane])['shifts']
        rotate = rot_mx(theta * np.pi/180)
        transformed = np.empty(data.shape)
        for i in range(data.shape[0]): 
            this = data[i,0,:,:]
            shift = shifts[i,0,:]
            affine = transl_mx((data.shape[-1]//2,)*2) @ (rotate @ transl_mx(shift - data.shape[-1]//2))
            transformed[i,0,:,:] = affine_transform(this, affine)
        fl.save(output_dir/"{:04d}.h5".format(plane), dict(stack_4D=transformed), compression="blosc")

In [5]:
rotated = SplitDataset(output_dir)
make_anatomy(rotated, n_jobs=1)

In [6]:
mask = io.imread(path/"mask.tif", plugin='tifffile')
transformed = np.empty(mask.shape)
rot_theta = fl.load(path/"rot_theta.h5")
for plane in range(mask.shape[0]):
    this = mask[plane,:,:]
    theta = rot_theta[plane]
    if theta == 0:
        transformed[plane,:,:] = this
    else:
        rotate = rot_mx(theta * np.pi/180)
        affine = transl_mx((mask.shape[-1]//2,)*2) @ (rotate @ transl_mx((-mask.shape[-1]//2,)*2))
        transformed[plane,:,:] = affine_transform(this, affine)
io.imsave(path/"mask_rotated.tif", transformed.astype("float32"), plugin='tifffile')